In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if len(gpus) > 0:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')

2023-11-09 15:45:06.914349: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 15:45:06.914391: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 15:45:06.914412: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 15:45:06.921866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 15:45:09.876624: I tensorflow/compiler/

Data processing

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pywt
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing 

In [3]:
!pip install PyWavelets

Defaulting to user installation because normal site-packages is not writeable


In [4]:
def read_data_csv(data_path):
    df=pd.read_csv(data_path)
    return df

In [5]:
def fill_missing_values(df):
    for column in df.columns:
        df[column] = df[column].interpolate()

    df.fillna(method='ffill',inplace=True)
    df.fillna(method='bfill',inplace=True)
    return df

In [6]:
def train_test_split_by_column(df,column_split,target):
    X_train_df=pd.DataFrame()
    X_test_df=pd.DataFrame()
    
    for city in df[column_split].unique():
      df_temp=df.loc[df[column_split]==city]
      threshold=int(df_temp.shape[0]*0.8)
    
      train_temp=df_temp[:threshold]
      test_temp=df_temp[threshold:]
    
      X_train_df=pd.concat([X_train_df,train_temp],axis=0)
      X_test_df=pd.concat([X_test_df,test_temp],axis=0)
    
    y_train_df=X_train_df[target].values
    y_test_df=X_test_df[target].values
    
    X_train_df.set_index((i for i in range(len(X_train_df))),inplace=True)
    X_test_df.set_index((i for i in range(len(X_test_df))),inplace=True)
    
    X_train_df.drop([target],axis=1,inplace=True)
    X_test_df.drop([target],axis=1,inplace=True)
    return X_train_df, y_train_df, X_test_df, y_test_df

In [7]:
def label_encoding(df,column):
    label_encoder = preprocessing.LabelEncoder()    
    # Encode labels in column 'species'. 
    df[column]= label_encoder.fit_transform(df[column]) 
    return df

In [8]:
def check_missing_values_percent(df):
    for col in df.columns:
      num_of_nan = df[col].isna().sum()
      percent_of_nan=num_of_nan*100/len(df)
      print(f"Column \"{col}\" has {num_of_nan} missing values ({percent_of_nan}%)")

In [9]:
# #Scale dữ liệu về đoạn 0-1
def scale_data(df_train,df_test,list_scale_features):
    scaler = MinMaxScaler(feature_range=(0, 1))
    values_train=df_train[list_scale_features].values
    scaled_values_train = scaler.fit_transform(values_train)
    df_train[list_scale_features]=scaled_values_train 
    
    values_test=df_test[list_scale_features].values
    scaled_values_test = scaler.transform(values_test)
    df_test[list_scale_features]=scaled_values_test

    return df_train,df_test

In [10]:
def wavelet_decomposition(data, wavelet_type='db1'):
    return pywt.wavedec(data, wavelet_type, mode='per')

def wavelet_reconstruction(coeffs, wavelet_type='db1'):
    return pywt.waverec(coeffs, wavelet_type, mode='per')

In [11]:
def create_dataset(dataset):
    global look_back
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        dataX.append(dataset[i:(i + look_back), 0])
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [12]:
def window_slide(train,label):
  #drop dữ liệu dư
  window_size = 24
  X = []
  Y = []
  label_index=window_size-1
  for city in train['City'].unique():
      df_city=train[train['City']==city]
      for i in range(window_size, len(df_city)):
        label_index+=1
        X.append(df_city.iloc[i-window_size:i,:].values)
        Y.append(label[label_index,:])
  return np.array(X),np.array(Y)

In [13]:
def build_lstm_autoencoder_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(units = 64,activation='relu'), input_shape=(X_train_wt.shape[1], X_train_wt.shape[-1])))
    model.add(RepeatVector(input_shape[0]))
    model.add(Bidirectional(LSTM(units = 64, activation='relu',return_sequences=True)))
    model.add(Dense(1))
    return model

In [14]:
def build_lstm_attention_model(input_shape):
    input = Input(shape=input_shape)
    lstm1=Bidirectional(LSTM(units = 64,activation='relu',return_sequences=True))(input)
    lstm_output=Bidirectional(LSTM(units = 32, activation='relu',return_sequences=True))(lstm1)
    # Attention mechanism
    attention_output = Attention()([lstm_output, lstm_output])
    # Concatenate LSTM output and attention output
    combined_output = Concatenate()([lstm_output, attention_output])
    # Additional layers if needed
    output = Dense(1)(combined_output)
    model = Model(inputs=[input], outputs=output)
    return model

In [15]:
def train_model(X_train,y_train,lr=0.001,epochs=10,batch_size=64):
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mse')
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    return model

In [16]:
#read dataset
df=read_data_csv('India_dataset_raw.csv')
df.head()

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,Country
0,Ahmedabad,1/1/2015 1:00,NaN,NaN,1.00,40.01,36.37,NaN,1.00,122.07,NaN,0.0,0.0,0.0,NaN,NaN,India
1,Ahmedabad,1/1/2015 2:00,NaN,NaN,0.02,27.75,19.73,NaN,0.02,85.90,NaN,0.0,0.0,0.0,NaN,NaN,India
2,Ahmedabad,1/1/2015 3:00,NaN,NaN,0.08,19.32,11.08,NaN,0.08,52.83,NaN,0.0,0.0,0.0,NaN,NaN,India
3,Ahmedabad,1/1/2015 4:00,NaN,NaN,0.30,16.45,9.20,NaN,0.30,39.53,153.58,0.0,0.0,0.0,NaN,NaN,India
4,Ahmedabad,1/1/2015 5:00,NaN,NaN,0.12,14.90,7.85,NaN,0.12,32.63,NaN,0.0,0.0,0.0,NaN,NaN,India


In [ ]:
#Kiểm tra xem bao nhiêu phần trăm dữ liệu bị thiếu của mỗi feature trong từng thành phố
for i in df['City'].unique():
    temp=df[df['City']==i]
    print(f'{i}:\n{(temp.isna().sum()/len(temp))*100}\n') #Đơn vị phần trăm

In [18]:
#Drop những thành phố có tỉ lệ missing value cao
for city in ['Ahmedabad','Patna','Lucknow','Jorapokhar','Gurugram','Chennai']:
    df.drop(df.index[df['City']==city],inplace=True)

#Drop những feature không liên quan đến chỉ số AQI
df = df.drop(['Country','AQI_Bucket','Benzene','Toluene','Xylene','Datetime'],axis=1)
df.head()

,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,AQI
48192,Aizawl,42.00,51.28,4.27,0.97,6.66,19.88,0.37,3.35,NaN,NaN
48193,Aizawl,41.17,49.96,4.51,1.27,7.24,21.55,0.38,3.44,NaN,NaN
48194,Aizawl,24.97,42.04,7.25,5.45,14.56,20.25,0.50,3.93,6.95,NaN
48195,Aizawl,26.95,38.86,7.31,2.52,12.13,21.94,0.52,3.93,NaN,NaN
48196,Aizawl,17.42,37.15,7.25,1.58,11.14,25.71,0.49,4.36,NaN,NaN


In [19]:
#Fill missing value
df=fill_missing_values(df)
    
#Kiểm tra lại số missing value   
check_missing_values_percent(df)

/tmp/ipykernel_1179928/3008641224.py:3: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[column] = df[column].interpolate()


Column "City" has 0 missing values (0.0%)
Column "PM2.5" has 0 missing values (0.0%)
Column "PM10" has 0 missing values (0.0%)
Column "NO" has 0 missing values (0.0%)
Column "NO2" has 0 missing values (0.0%)
Column "NOx" has 0 missing values (0.0%)
Column "NH3" has 0 missing values (0.0%)
Column "CO" has 0 missing values (0.0%)
Column "SO2" has 0 missing values (0.0%)
Column "O3" has 0 missing values (0.0%)
Column "AQI" has 0 missing values (0.0%)


/tmp/ipykernel_1179928/3008641224.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill',inplace=True)
/tmp/ipykernel_1179928/3008641224.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill',inplace=True)


In [28]:
#Tách dữ liệu thành tập train và test
X_train_df, y_train_df, X_test_df, y_test_df=train_test_split_by_column(df,'City','AQI')
print('Number of X train city: ',X_train_df['City'].nunique())
print('Number of X test city: ',X_test_df['City'].nunique())
print('X train: ',X_train_df.shape)
print('y train: ',y_train_df.shape)
print('X test: ',X_test_df.shape)
print('y test: ',y_test_df.shape)

Number of X train city:  20
Number of X test city:  20
X train:  (360361, 10)
y train:  (360361,)
X test:  (90101, 10)
y test:  (90101,)


In [21]:
df_train_new=label_encoding(X_train_df,'City')
df_test_new=label_encoding(X_test_df,'City')
df_train_new

,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3
0,0,42.00,51.28,4.27,0.97,6.66,19.88,0.37,3.35,6.9500
1,0,41.17,49.96,4.51,1.27,7.24,21.55,0.38,3.44,6.9500
2,0,24.97,42.04,7.25,5.45,14.56,20.25,0.50,3.93,6.9500
3,0,26.95,38.86,7.31,2.52,12.13,21.94,0.52,3.93,5.7975
4,0,17.42,37.15,7.25,1.58,11.14,25.71,0.49,4.36,4.6450
...,...,...,...,...,...,...,...,...,...,...
360356,19,42.75,116.00,13.72,43.15,34.10,4.62,1.98,5.55,31.8200
360357,19,43.00,127.00,23.23,58.55,50.00,4.10,2.12,9.50,18.6200
360358,19,39.50,138.50,34.90,57.93,59.17,3.70,2.01,11.07,14.0700
360359,19,33.00,117.00,41.33,52.92,61.73,3.72,2.67,11.73,12.6000


In [22]:
scaled_features=['PM2.5','PM10','SO2','CO','O3','NO','NO2','NOx','NH3']
df_train_scaled,df_test_scaled=scale_data(df_train_new,df_test_new,scaled_features)
df_train_scaled.head()

,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3
0,0,0.041991,0.051271,0.008538,0.001922,0.013462,0.040901,0.007406,0.016707,0.013947
1,0,0.041161,0.049950,0.009019,0.002523,0.014634,0.044338,0.007606,0.017157,0.013947
2,0,0.024960,0.042030,0.014510,0.010891,0.029430,0.041662,0.010008,0.019608,0.013947
3,0,0.026941,0.038850,0.014630,0.005025,0.024518,0.045141,0.010408,0.019608,0.011631
4,0,0.017410,0.037140,0.014510,0.003143,0.022517,0.052901,0.009808,0.021759,0.009315


In [23]:
#Chia dữ liệu thành các frame
y_train=np.reshape(y_train_df,(len(y_train_df),1))
y_test=np.reshape(y_test_df,(len(y_test_df),1))

X_train,y_train=window_slide(df_train_new,y_train)
print('Y train' ,y_train.shape)
print('X train' ,X_train.shape)

X_test,y_test=window_slide(df_test_new,y_test)
print('Y test' ,y_test.shape)
print('X test' ,X_test.shape)

Y train (359881, 1)
X train (359881, 24, 10)
Y test (89621, 1)
X test (89621, 24, 10)


In [26]:
#Main model
from numpy import array
import keras
from keras.models import Sequential
from keras.layers import LSTM,Bidirectional
from keras.layers import Dense
from keras.layers import RepeatVector,Dropout
from keras.layers import TimeDistributed,BatchNormalization, Input,Attention,Concatenate
from keras.utils import plot_model
from tensorflow.keras import regularizers
from keras.models import Model
import torch
from keras.optimizers import Adam

model=build_lstm_autoencoder_model((X_train.shape[1], X_train.shape[-1]))
model=train_model([X_train], y_train)

2023-11-09 15:46:06.312293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-09 15:46:06.312601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-09 15:46:06.312809: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/10


2023-11-09 15:46:12.820987: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f70740805e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-09 15:46:12.821010: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-11-09 15:46:12.824589: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-09 15:46:12.866879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-11-09 15:46:12.953789: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 555/5624 [=>............................] - ETA: 9:11 - loss: 4118774.2500

KeyboardInterrupt: 

In [ ]:
import pickle
# save the model to disk
filename = 'wt_attention.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# import pickle
# # filename = 'finalized_model.sav'
# # model = pickle.load(open(filename, 'rb'))

# filename = 'wt_attention.sav'
# model = pickle.load(open(filename, 'rb'))

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred=model.predict(X_test_wt)
mse = mean_squared_error(y_test, y_pred)
print('MSE: ',mse)

In [ ]:
#Plot the graph between actual vs predicted values
plt.figure(figsize=(10,6))
plt.plot(y_pred[:1000,:], color= 'green',label = 'Predicted AQI')
plt.plot(y_test[:1000,:] , color = 'red',label = 'Actual AQI')
plt.title("AQI Prediction (Multivariate)")
plt.xlabel("Date")
plt.ylabel("AQI")
plt.legend()
plt.show()
plt.savefig('graph.png')